# AutoKaggle: Error Analysis and Debugging Workflow

## Mục Tiêu

Notebook này tập trung vào phân tích chuyên sâu về quy trình phân tích lỗi và debugging được giới thiệu trong bài báo "AutoKaggle: A Multi-Agent Framework for Autonomous Data Science Competitions". Chúng ta sẽ khám phá cách thiết kế và triển khai một hệ thống debug tự động có khả năng phát hiện, phân tích và sửa chữa lỗi trong quá trình phát triển code.

Sau khi hoàn thành notebook này, bạn sẽ:
1. Hiểu được các loại lỗi phổ biến trong quy trình khoa học dữ liệu
2. Nắm được quy trình debug tự động và iterative testing
3. Hiểu được cách phân tích và xử lý lỗi một cách có hệ thống
4. Có thể triển khai hệ thống debug tự động cho các dự án ML

## Trích Xuất Từ Bài Báo

> "AutoKaggle employs iterative debugging and testing to ensure the generated code meets requirements and is reliable. This process includes code execution, error detection, debugging, and unit testing." (Section 3.2 Development-Based Iterative Debugging and Testing, Page 5)

> "The iterative debugging process involves three main steps: 1) Execute the generated code and capture any runtime errors, 2) If errors are detected, analyze the error messages and identify the root cause, 3) Generate corrected code and test again until the code executes successfully." (Section 3.2 Development-Based Iterative Debugging and Testing, Page 5)

> "Unit testing is performed to validate the correctness of the generated code. Each phase has specific test cases designed to verify that the code produces the expected outputs and handles edge cases appropriately." (Section 3.2 Development-Based Iterative Debugging and Testing, Page 5)

> "The system tracks error patterns and maintains a history of common issues and their solutions. This knowledge base helps improve the debugging process over time by providing context-aware suggestions for similar errors." (Section 3.2 Development-Based Iterative Debugging and Testing, Page 5)

![Error Analysis Workflow](https://m-a-p.ai/AutoKaggle.github.io/assets/images/fig2.png)
*(Note: Diagram representation - actual image not included)*

## Cài Đặt Môi Trường

In [ ]:
!pip install -q pandas numpy matplotlib seaborn scikit-learn
!pip install -q langchain langchain-openai
!pip install -q ast traceback sys

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Optional, Tuple, Any, Union, Callable
from enum import Enum
import ast
import traceback
import sys
import io
import contextlib
import time
import re
import json
import warnings
from dataclasses import dataclass
from abc import ABC, abstractmethod
import logging

# Import cho LLM integration
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Thiết lập logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Thiết lập môi trường
warnings.filterwarnings('ignore')
plt.style.use('default')

## Phần 1: Định Nghĩa Các Loại Lỗi và Mức Độ Nghiêm Trọng

Trước khi xây dựng hệ thống debug, chúng ta cần phân loại các loại lỗi có thể gặp phải trong quy trình khoa học dữ liệu:

In [ ]:
class ErrorType(Enum):
    """Phân loại các loại lỗi trong quy trình khoa học dữ liệu"""
    SYNTAX_ERROR = "Syntax Error"
    IMPORT_ERROR = "Import Error"
    TYPE_ERROR = "Type Error"
    VALUE_ERROR = "Value Error"
    KEY_ERROR = "Key Error"
    INDEX_ERROR = "Index Error"
    ATTRIBUTE_ERROR = "Attribute Error"
    MEMORY_ERROR = "Memory Error"
    RUNTIME_ERROR = "Runtime Error"
    LOGIC_ERROR = "Logic Error"
    DATA_ERROR = "Data Error"
    MODEL_ERROR = "Model Error"
    PERFORMANCE_ERROR = "Performance Error"
    UNKNOWN_ERROR = "Unknown Error"

class ErrorSeverity(Enum):
    """Mức độ nghiêm trọng của lỗi"""
    CRITICAL = "Critical"  # Lỗi ngăn cản hoàn toàn việc thực thi
    HIGH = "High"         # Lỗi ảnh hưởng nghiêm trọng đến kết quả
    MEDIUM = "Medium"     # Lỗi có thể ảnh hưởng đến chất lượng
    LOW = "Low"           # Lỗi nhỏ, không ảnh hưởng nhiều
    WARNING = "Warning"   # Cảnh báo, không phải lỗi thực sự

@dataclass
class ErrorInfo:
    """Thông tin chi tiết về một lỗi"""
    error_type: ErrorType
    severity: ErrorSeverity
    message: str
    traceback: str
    line_number: Optional[int]
    code_snippet: Optional[str]
    suggested_fix: Optional[str]
    timestamp: float
    context: Dict[str, Any]

class ErrorClassifier:
    """Phân loại lỗi dựa trên thông tin exception"""
    
    def __init__(self):
        self.error_patterns = {
            ErrorType.SYNTAX_ERROR: [
                r"SyntaxError",
                r"invalid syntax",
                r"unexpected EOF",
                r"IndentationError"
            ],
            ErrorType.IMPORT_ERROR: [
                r"ImportError",
                r"ModuleNotFoundError",
                r"No module named"
            ],
            ErrorType.TYPE_ERROR: [
                r"TypeError",
                r"unsupported operand type",
                r"object is not callable",
                r"takes .* positional arguments"
            ],
            ErrorType.VALUE_ERROR: [
                r"ValueError",
                r"invalid literal",
                r"could not convert",
                r"empty sequence"
            ],
            ErrorType.KEY_ERROR: [
                r"KeyError",
                r"key not found"
            ],
            ErrorType.INDEX_ERROR: [
                r"IndexError",
                r"list index out of range",
                r"index out of bounds"
            ],
            ErrorType.ATTRIBUTE_ERROR: [
                r"AttributeError",
                r"has no attribute"
            ],
            ErrorType.MEMORY_ERROR: [
                r"MemoryError",
                r"out of memory",
                r"OutOfMemoryError"
            ],
            ErrorType.DATA_ERROR: [
                r"data type.*not understood",
                r"input contains NaN",
                r"input contains infinity",
                r"empty DataFrame",
                r"no columns to parse"
            ],
            ErrorType.MODEL_ERROR: [
                r"convergence",
                r"not fitted",
                r"singular matrix",
                r"ill-conditioned"
            ]
        }
        
        self.severity_patterns = {
            ErrorSeverity.CRITICAL: [
                r"SyntaxError", r"ImportError", r"MemoryError"
            ],
            ErrorSeverity.HIGH: [
                r"TypeError", r"ValueError", r"AttributeError"
            ],
            ErrorSeverity.MEDIUM: [
                r"KeyError", r"IndexError", r"RuntimeWarning"
            ],
            ErrorSeverity.LOW: [
                r"FutureWarning", r"DeprecationWarning"
            ]
        }
    
    def classify_error(self, error_message: str, traceback_str: str) -> Tuple[ErrorType, ErrorSeverity]:
        """Phân loại lỗi dựa trên error message và traceback"""
        combined_text = f"{error_message} {traceback_str}"
        
        # Xác định loại lỗi
        error_type = ErrorType.UNKNOWN_ERROR
        for etype, patterns in self.error_patterns.items():
            for pattern in patterns:
                if re.search(pattern, combined_text, re.IGNORECASE):
                    error_type = etype
                    break
            if error_type != ErrorType.UNKNOWN_ERROR:
                break
        
        # Xác định mức độ nghiêm trọng
        severity = ErrorSeverity.MEDIUM  # Mặc định
        for sev, patterns in self.severity_patterns.items():
            for pattern in patterns:
                if re.search(pattern, combined_text, re.IGNORECASE):
                    severity = sev
                    break
            if severity != ErrorSeverity.MEDIUM:
                break
        
        return error_type, severity
    
    def extract_line_number(self, traceback_str: str) -> Optional[int]:
        """Trích xuất số dòng từ traceback"""
        line_pattern = r"line (\d+)"
        matches = re.findall(line_pattern, traceback_str)
        if matches:
            return int(matches[-1])  # Lấy dòng cuối cùng (thường là dòng gây lỗi)
        return None
    
    def extract_code_snippet(self, code: str, line_number: Optional[int], context_lines: int = 3) -> Optional[str]:
        """Trích xuất đoạn code xung quanh dòng lỗi"""
        if line_number is None:
            return None
        
        lines = code.split('\n')
        start = max(0, line_number - context_lines - 1)
        end = min(len(lines), line_number + context_lines)
        
        snippet_lines = []
        for i in range(start, end):
            marker = ">>> " if i == line_number - 1 else "    "
            snippet_lines.append(f"{marker}{i+1}: {lines[i]}")
        
        return "\n".join(snippet_lines)

# Khởi tạo classifier
error_classifier = ErrorClassifier()

## Phần 2: Hệ Thống Thực Thi Code An Toàn

Để debug hiệu quả, chúng ta cần một hệ thống có thể thực thi code an toàn và capture tất cả các lỗi:

In [ ]:
class SafeCodeExecutor:
    """Hệ thống thực thi code an toàn với error capturing"""
    
    def __init__(self, timeout: int = 30):
        self.timeout = timeout
        self.execution_history = []
        self.global_namespace = {
            '__builtins__': __builtins__,
            'pd': pd,
            'np': np,
            'plt': plt,
            'sns': sns
        }
    
    def execute_code(self, code: str, context: Dict[str, Any] = None) -> Dict[str, Any]:
        """
        Thực thi code và capture kết quả/lỗi
        
        Parameters:
        - code: Code Python cần thực thi
        - context: Context variables để inject vào namespace
        
        Returns:
        - Dictionary chứa kết quả thực thi
        """
        execution_result = {
            'success': False,
            'output': '',
            'error': None,
            'error_info': None,
            'execution_time': 0,
            'variables_created': [],
            'timestamp': time.time()
        }
        
        # Chuẩn bị namespace
        local_namespace = self.global_namespace.copy()
        if context:
            local_namespace.update(context)
        
        # Capture variables trước khi thực thi
        vars_before = set(local_namespace.keys())
        
        # Capture stdout
        old_stdout = sys.stdout
        sys.stdout = captured_output = io.StringIO()
        
        start_time = time.time()
        
        try:
            # Kiểm tra syntax trước
            try:
                ast.parse(code)
            except SyntaxError as e:
                raise SyntaxError(f"Syntax error at line {e.lineno}: {e.msg}")
            
            # Thực thi code
            exec(code, local_namespace)
            
            execution_result['success'] = True
            execution_result['output'] = captured_output.getvalue()
            
            # Xác định variables mới được tạo
            vars_after = set(local_namespace.keys())
            execution_result['variables_created'] = list(vars_after - vars_before)
            
            # Cập nhật global namespace
            self.global_namespace.update(local_namespace)
            
        except Exception as e:
            # Capture error information
            error_message = str(e)
            traceback_str = traceback.format_exc()
            
            # Phân loại lỗi
            error_type, severity = error_classifier.classify_error(error_message, traceback_str)
            line_number = error_classifier.extract_line_number(traceback_str)
            code_snippet = error_classifier.extract_code_snippet(code, line_number)
            
            # Tạo ErrorInfo object
            error_info = ErrorInfo(
                error_type=error_type,
                severity=severity,
                message=error_message,
                traceback=traceback_str,
                line_number=line_number,
                code_snippet=code_snippet,
                suggested_fix=None,  # Sẽ được fill bởi FixSuggester
                timestamp=time.time(),
                context=context or {}
            )
            
            execution_result['error'] = error_message
            execution_result['error_info'] = error_info
        
        finally:
            # Khôi phục stdout
            sys.stdout = old_stdout
            execution_result['execution_time'] = time.time() - start_time
        
        # Lưu vào history
        self.execution_history.append({
            'code': code,
            'result': execution_result
        })
        
        return execution_result
    
    def get_variable(self, var_name: str) -> Any:
        """Lấy giá trị của một variable từ namespace"""
        return self.global_namespace.get(var_name)
    
    def set_variable(self, var_name: str, value: Any) -> None:
        """Set giá trị cho một variable trong namespace"""
        self.global_namespace[var_name] = value
    
    def clear_namespace(self) -> None:
        """Clear các variables (giữ lại built-ins và imports cơ bản)"""
        basic_vars = {
            '__builtins__': __builtins__,
            'pd': pd,
            'np': np,
            'plt': plt,
            'sns': sns
        }
        self.global_namespace = basic_vars
    
    def get_execution_stats(self) -> Dict[str, Any]:
        """Thống kê về quá trình thực thi"""
        if not self.execution_history:
            return {'total_executions': 0, 'success_rate': 0, 'average_time': 0}
        
        total = len(self.execution_history)
        successful = sum(1 for entry in self.execution_history if entry['result']['success'])
        total_time = sum(entry['result']['execution_time'] for entry in self.execution_history)
        
        return {
            'total_executions': total,
            'successful_executions': successful,
            'failed_executions': total - successful,
            'success_rate': successful / total * 100,
            'average_time': total_time / total,
            'total_time': total_time
        }

# Khởi tạo safe executor
safe_executor = SafeCodeExecutor()

## Phần 3: Hệ Thống Đề Xuất Sửa Lỗi Tự Động

Đây là thành phần quan trọng nhất - hệ thống AI có thể phân tích lỗi và đề xuất cách sửa:

In [ ]:
class ErrorFixSuggester:
    """Hệ thống đề xuất cách sửa lỗi sử dụng LLM"""
    
    def __init__(self, model_name: str = "gpt-3.5-turbo"):
        self.model = ChatOpenAI(model=model_name, temperature=0.1)
        self.fix_history = []  # Lịch sử các lần fix
        self.common_fixes = self._load_common_fixes()
        
        # Template cho error analysis
        self.analysis_prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content="""
            You are an expert Python debugger and data science code analyst. Your task is to analyze errors and provide specific, actionable fixes.
            
            Follow this analysis framework:
            1. ROOT CAUSE ANALYSIS: Identify the exact cause of the error
            2. IMPACT ASSESSMENT: Determine how this error affects the overall workflow
            3. FIX STRATEGY: Provide a specific solution
            4. ALTERNATIVE APPROACHES: Suggest alternative methods if applicable
            5. PREVENTION: How to avoid this error in the future
            
            Focus on data science and machine learning context. Provide working code fixes.
            """),
            HumanMessage(content="""
            Analyze this error and provide a fix:
            
            ERROR TYPE: {error_type}
            SEVERITY: {severity}
            ERROR MESSAGE: {error_message}
            
            CODE THAT FAILED:
            ```python
            {code_snippet}
            ```
            
            FULL TRACEBACK:
            {traceback}
            
            CONTEXT: {context}
            
            Please provide:
            1. A clear explanation of what went wrong
            2. The corrected code
            3. Alternative approaches if applicable
            4. How to prevent this error in the future
            """)
        ])
        
        self.chain = self.analysis_prompt | self.model | StrOutputParser()
    
    def _load_common_fixes(self) -> Dict[ErrorType, List[str]]:
        """Load common fixes for frequent error types"""
        return {
            ErrorType.IMPORT_ERROR: [
                "Install missing package with pip install",
                "Check package name spelling",
                "Import from correct module path"
            ],
            ErrorType.KEY_ERROR: [
                "Check if key exists before accessing",
                "Use .get() method with default value",
                "Verify dataframe column names"
            ],
            ErrorType.TYPE_ERROR: [
                "Check data types of variables",
                "Convert to appropriate type",
                "Verify function arguments"
            ],
            ErrorType.VALUE_ERROR: [
                "Check input data validity",
                "Handle missing or invalid values",
                "Verify data ranges and constraints"
            ],
            ErrorType.DATA_ERROR: [
                "Clean data before processing",
                "Handle NaN and infinite values",
                "Check data schema and types"
            ]
        }
    
    def suggest_fix(self, error_info: ErrorInfo, original_code: str) -> Dict[str, Any]:
        """
        Đề xuất cách sửa lỗi
        
        Parameters:
        - error_info: Thông tin lỗi đã được phân tích
        - original_code: Code gốc gây lỗi
        
        Returns:
        - Dictionary chứa các đề xuất sửa lỗi
        """
        start_time = time.time()
        
        # Kiểm tra xem có fix nhanh không
        quick_fix = self._try_quick_fix(error_info, original_code)
        
        # Sử dụng LLM để phân tích sâu
        llm_analysis = self._get_llm_analysis(error_info, original_code)
        
        # Tạo suggested fixes
        suggested_fixes = {
            'quick_fix': quick_fix,
            'llm_analysis': llm_analysis,
            'common_fixes': self.common_fixes.get(error_info.error_type, []),
            'analysis_time': time.time() - start_time,
            'confidence': self._calculate_confidence(error_info, quick_fix, llm_analysis)
        }
        
        # Lưu vào history
        self.fix_history.append({
            'error_info': error_info,
            'original_code': original_code,
            'suggested_fixes': suggested_fixes,
            'timestamp': time.time()
        })
        
        return suggested_fixes
    
    def _try_quick_fix(self, error_info: ErrorInfo, code: str) -> Optional[str]:
        """Thử các quick fix patterns cho lỗi phổ biến"""
        error_type = error_info.error_type
        message = error_info.message.lower()
        
        # Import error fixes
        if error_type == ErrorType.IMPORT_ERROR:
            if "no module named" in message:
                module_name = re.search(r"no module named ['\"]([^'\"]+)['\"], message)
                if module_name:
                    return f"# Install missing module\n!pip install {module_name.group(1)}\n\n{code}"
        
        # KeyError fixes
        elif error_type == ErrorType.KEY_ERROR:
            if "'" in message:  # Extract key name
                key_match = re.search(r"'([^']+)'", message)
                if key_match:
                    key_name = key_match.group(1)
                    # Suggest using .get() method
                    lines = code.split('\n')
                    fixed_lines = []
                    for line in lines:
                        if f"['{key_name}']" in line or f'["{key_name}"]' in line:
                            # Replace with .get() method
                            fixed_line = re.sub(
                                r"\[['\"]" + re.escape(key_name) + r"['\"]\]",
                                f".get('{key_name}', None)",
                                line
                            )
                            fixed_lines.append(fixed_line)
                        else:
                            fixed_lines.append(line)
                    return '\n'.join(fixed_lines)
        
        # Type error fixes
        elif error_type == ErrorType.TYPE_ERROR:
            if "unsupported operand type" in message:
                # Suggest type conversion
                return f"# Add type conversion\n# Check data types before operations\n{code}"
        
        return None
    
    def _get_llm_analysis(self, error_info: ErrorInfo, code: str) -> str:
        """Sử dụng LLM để phân tích lỗi và đề xuất fix"""
        try:
            # Chuẩn bị context
            context_str = json.dumps(error_info.context, indent=2) if error_info.context else "No additional context"
            
            # Gọi LLM
            analysis = self.chain.invoke({
                'error_type': error_info.error_type.value,
                'severity': error_info.severity.value,
                'error_message': error_info.message,
                'code_snippet': error_info.code_snippet or code,
                'traceback': error_info.traceback,
                'context': context_str
            })
            
            return analysis
            
        except Exception as e:
            return f"Error during LLM analysis: {str(e)}"
    
    def _calculate_confidence(self, error_info: ErrorInfo, quick_fix: Optional[str], llm_analysis: str) -> float:
        """Tính confidence score cho fix suggestion"""
        confidence = 0.5  # Base confidence
        
        # Tăng confidence nếu có quick fix
        if quick_fix:
            confidence += 0.2
        
        # Tăng confidence dựa trên error type
        if error_info.error_type in [ErrorType.IMPORT_ERROR, ErrorType.SYNTAX_ERROR]:
            confidence += 0.2
        
        # Tăng confidence nếu LLM analysis có code block
        if "```python" in llm_analysis:
            confidence += 0.1
        
        return min(confidence, 1.0)
    
    def extract_fixed_code_from_analysis(self, llm_analysis: str) -> Optional[str]:
        """Trích xuất code đã fix từ LLM analysis"""
        # Tìm code blocks trong analysis
        code_pattern = r"```python\s*\n(.*?)\n\s*```"
        matches = re.findall(code_pattern, llm_analysis, re.DOTALL)
        
        if matches:
            # Lấy code block đầu tiên (thường là fixed code)
            return matches[0].strip()
        
        return None
    
    def get_fix_success_rate(self) -> Dict[str, float]:
        """Thống kê tỷ lệ thành công của các loại fix"""
        if not self.fix_history:
            return {}
        
        success_by_type = {}
        total_by_type = {}
        
        for entry in self.fix_history:
            error_type = entry['error_info'].error_type.value
            total_by_type[error_type] = total_by_type.get(error_type, 0) + 1
            
            # Check if fix was successful (simplified - would need actual validation)
            confidence = entry['suggested_fixes']['confidence']
            if confidence > 0.7:  # Consider high confidence as successful
                success_by_type[error_type] = success_by_type.get(error_type, 0) + 1
        
        success_rates = {}
        for error_type in total_by_type:
            success_rates[error_type] = success_by_type.get(error_type, 0) / total_by_type[error_type]
        
        return success_rates

# Khởi tạo fix suggester (comment để tránh lỗi API key)
# fix_suggester = ErrorFixSuggester()
print("Error Fix Suggester class defined (API key required for instantiation)")

## Phần 4: Unit Testing Framework

Hệ thống unit testing để verify tính đúng đắn của code đã fix:

In [ ]:
class TestCase:
    """Định nghĩa một test case"""
    
    def __init__(self, name: str, test_code: str, expected_result: Any = None, 
                 validation_func: Callable = None, description: str = ""):
        self.name = name
        self.test_code = test_code
        self.expected_result = expected_result
        self.validation_func = validation_func
        self.description = description
    
    def __repr__(self):
        return f"TestCase(name='{self.name}', description='{self.description}')"

class TestResult:
    """Kết quả của một test case"""
    
    def __init__(self, test_case: TestCase, passed: bool, 
                 actual_result: Any = None, error: str = None, 
                 execution_time: float = 0):
        self.test_case = test_case
        self.passed = passed
        self.actual_result = actual_result
        self.error = error
        self.execution_time = execution_time
        self.timestamp = time.time()

class UnitTestFramework:
    """Framework for running unit tests on code"""
    
    def __init__(self, executor: SafeCodeExecutor):
        self.executor = executor
        self.test_suites = {}
        self.test_results = []
    
    def create_test_suite(self, suite_name: str, test_cases: List[TestCase]):
        """Tạo một test suite"""
        self.test_suites[suite_name] = test_cases
    
    def add_test_case(self, suite_name: str, test_case: TestCase):
        """Thêm test case vào suite"""
        if suite_name not in self.test_suites:
            self.test_suites[suite_name] = []
        self.test_suites[suite_name].append(test_case)
    
    def run_test_case(self, test_case: TestCase, context: Dict[str, Any] = None) -> TestResult:
        """Chạy một test case đơn lẻ"""
        start_time = time.time()
        
        try:
            # Thực thi test code
            execution_result = self.executor.execute_code(test_case.test_code, context)
            execution_time = time.time() - start_time
            
            if not execution_result['success']:
                # Test failed due to execution error
                return TestResult(
                    test_case=test_case,
                    passed=False,
                    error=execution_result['error'],
                    execution_time=execution_time
                )
            
            # Kiểm tra kết quả
            actual_result = execution_result.get('output', '')
            
            # Sử dụng validation function nếu có
            if test_case.validation_func:
                try:
                    passed = test_case.validation_func(actual_result, self.executor.global_namespace)
                except Exception as e:
                    return TestResult(
                        test_case=test_case,
                        passed=False,
                        error=f"Validation function error: {str(e)}",
                        execution_time=execution_time
                    )
            else:
                # So sánh với expected result
                if test_case.expected_result is not None:
                    passed = str(actual_result).strip() == str(test_case.expected_result).strip()
                else:
                    # Nếu không có expected result, chỉ cần code chạy thành công
                    passed = True
            
            return TestResult(
                test_case=test_case,
                passed=passed,
                actual_result=actual_result,
                execution_time=execution_time
            )
            
        except Exception as e:
            execution_time = time.time() - start_time
            return TestResult(
                test_case=test_case,
                passed=False,
                error=str(e),
                execution_time=execution_time
            )
    
    def run_test_suite(self, suite_name: str, context: Dict[str, Any] = None) -> List[TestResult]:
        """Chạy toàn bộ test suite"""
        if suite_name not in self.test_suites:
            raise ValueError(f"Test suite '{suite_name}' not found")
        
        results = []
        test_cases = self.test_suites[suite_name]
        
        print(f"Running test suite: {suite_name}")
        print(f"Total test cases: {len(test_cases)}")
        print("=" * 50)
        
        for i, test_case in enumerate(test_cases, 1):
            print(f"Running test {i}/{len(test_cases)}: {test_case.name}")
            
            result = self.run_test_case(test_case, context)
            results.append(result)
            
            # Log kết quả
            status = "PASS" if result.passed else "FAIL"
            print(f"  {status} - {result.execution_time:.3f}s")
            
            if not result.passed and result.error:
                print(f"  Error: {result.error}")
        
        # Tổng kết
        passed_count = sum(1 for r in results if r.passed)
        total_count = len(results)
        success_rate = passed_count / total_count * 100
        
        print("=" * 50)
        print(f"Test Results: {passed_count}/{total_count} passed ({success_rate:.1f}%)")
        
        # Lưu kết quả
        self.test_results.extend(results)
        
        return results
    
    def create_data_science_test_suite(self, phase: str) -> List[TestCase]:
        """Tạo test suite cho các phase khác nhau của data science"""
        if phase == "data_loading":
            return [
                TestCase(
                    name="test_data_loading",
                    test_code="assert 'df' in globals(), 'DataFrame df should be created'",
                    description="Check if data is loaded into df variable",
                    validation_func=lambda output, namespace: 'df' in namespace and hasattr(namespace['df'], 'shape')
                ),
                TestCase(
                    name="test_data_not_empty",
                    test_code="assert df.shape[0] > 0, 'DataFrame should not be empty'",
                    description="Check if loaded data is not empty",
                    validation_func=lambda output, namespace: namespace.get('df') is not None and len(namespace['df']) > 0
                )
            ]
        
        elif phase == "data_cleaning":
            return [
                TestCase(
                    name="test_no_infinite_values",
                    test_code="assert not np.isinf(df.select_dtypes(include=[np.number]).values).any(), 'No infinite values should remain'",
                    description="Check for infinite values in numeric columns"
                ),
                TestCase(
                    name="test_data_types_valid",
                    test_code="assert all(df.dtypes.notna()), 'All columns should have valid data types'",
                    description="Verify data types are valid"
                )
            ]
        
        elif phase == "feature_engineering":
            return [
                TestCase(
                    name="test_features_created",
                    test_code="original_cols = globals().get('original_columns', [])\nassert len(df.columns) >= len(original_cols), 'New features should be created'",
                    description="Check if new features were created"
                ),
                TestCase(
                    name="test_no_null_targets",
                    test_code="target_col = globals().get('target_column')\nif target_col and target_col in df.columns:\n    assert df[target_col].notna().all(), 'Target column should not have null values'",
                    description="Check target column validity"
                )
            ]
        
        elif phase == "model_training":
            return [
                TestCase(
                    name="test_model_created",
                    test_code="assert 'model' in globals(), 'Model should be created'",
                    description="Check if model variable exists",
                    validation_func=lambda output, namespace: 'model' in namespace
                ),
                TestCase(
                    name="test_model_fitted",
                    test_code="assert hasattr(model, 'predict'), 'Model should have predict method'",
                    description="Check if model is fitted and ready for prediction"
                )
            ]
        
        else:
            return [
                TestCase(
                    name="test_basic_execution",
                    test_code="pass  # Basic execution test",
                    description="Basic test to ensure code executes without errors"
                )
            ]
    
    def generate_test_report(self) -> pd.DataFrame:
        """Tạo báo cáo chi tiết về test results"""
        if not self.test_results:
            return pd.DataFrame()
        
        report_data = []
        for result in self.test_results:
            report_data.append({
                'Test Name': result.test_case.name,
                'Description': result.test_case.description,
                'Status': 'PASS' if result.passed else 'FAIL',
                'Execution Time (s)': result.execution_time,
                'Error': result.error if result.error else '',
                'Timestamp': pd.to_datetime(result.timestamp, unit='s')
            })
        
        return pd.DataFrame(report_data)

# Khởi tạo test framework
test_framework = UnitTestFramework(safe_executor)

## Phần 5: Iterative Debugging Engine

Đây là engine chính kết hợp tất cả các thành phần để thực hiện iterative debugging:

In [ ]:
class IterativeDebuggingEngine:
    """Engine chính cho iterative debugging và testing"""
    
    def __init__(self, safe_executor: SafeCodeExecutor, 
                 test_framework: UnitTestFramework,
                 fix_suggester: Optional[ErrorFixSuggester] = None,
                 max_iterations: int = 5):
        self.executor = safe_executor
        self.test_framework = test_framework
        self.fix_suggester = fix_suggester
        self.max_iterations = max_iterations
        self.debugging_sessions = []
    
    def debug_code(self, code: str, phase: str = "general", 
                   context: Dict[str, Any] = None,
                   custom_tests: List[TestCase] = None) -> Dict[str, Any]:
        """
        Thực hiện iterative debugging cho một đoạn code
        
        Parameters:
        - code: Code cần debug
        - phase: Phase của data science workflow
        - context: Context variables
        - custom_tests: Custom test cases
        
        Returns:
        - Dictionary chứa kết quả debugging
        """
        session_start = time.time()
        session_id = f"debug_session_{int(session_start)}"
        
        debugging_session = {
            'session_id': session_id,
            'original_code': code,
            'phase': phase,
            'context': context or {},
            'iterations': [],
            'final_code': None,
            'success': False,
            'total_time': 0
        }
        
        current_code = code
        iteration = 0
        
        print(f"Starting debugging session: {session_id}")
        print(f"Phase: {phase}")
        print("=" * 60)
        
        while iteration < self.max_iterations:
            iteration += 1
            iteration_start = time.time()
            
            print(f"\nIteration {iteration}/{self.max_iterations}")
            print("-" * 30)
            
            # Step 1: Execute code
            print("Step 1: Executing code...")
            execution_result = self.executor.execute_code(current_code, context)
            
            iteration_data = {
                'iteration': iteration,
                'code': current_code,
                'execution_result': execution_result,
                'test_results': None,
                'fix_suggestions': None,
                'iteration_time': 0
            }
            
            if execution_result['success']:
                print("✓ Code executed successfully")
                
                # Step 2: Run unit tests
                print("Step 2: Running unit tests...")
                test_results = self._run_tests(phase, context, custom_tests)
                iteration_data['test_results'] = test_results
                
                passed_tests = sum(1 for r in test_results if r.passed)
                total_tests = len(test_results)
                
                print(f"Test results: {passed_tests}/{total_tests} passed")
                
                if passed_tests == total_tests:
                    print("🎉 All tests passed! Debugging successful.")
                    debugging_session['success'] = True
                    debugging_session['final_code'] = current_code
                    break
                else:
                    print("⚠️ Some tests failed. Analyzing failures...")
                    # Tạo error info từ failed tests
                    failed_tests = [r for r in test_results if not r.passed]
                    error_info = self._create_error_info_from_tests(failed_tests)
            else:
                print("✗ Code execution failed")
                error_info = execution_result['error_info']
                print(f"Error: {error_info.message}")
            
            # Step 3: Get fix suggestions
            if not execution_result['success'] or (iteration_data['test_results'] and 
                                                   not all(r.passed for r in iteration_data['test_results'])):
                print("Step 3: Generating fix suggestions...")
                
                if self.fix_suggester:
                    fix_suggestions = self.fix_suggester.suggest_fix(error_info, current_code)
                    iteration_data['fix_suggestions'] = fix_suggestions
                    
                    # Try to extract fixed code
                    fixed_code = self._extract_best_fix(fix_suggestions, current_code)
                    if fixed_code:
                        current_code = fixed_code
                        print("✓ Applied suggested fix")
                    else:
                        print("⚠️ Could not extract actionable fix")
                        break
                else:
                    print("⚠️ No fix suggester available")
                    break
            
            iteration_data['iteration_time'] = time.time() - iteration_start
            debugging_session['iterations'].append(iteration_data)
        
        debugging_session['total_time'] = time.time() - session_start
        
        # Final summary
        print("\n" + "=" * 60)
        print("DEBUGGING SESSION SUMMARY")
        print("=" * 60)
        print(f"Session ID: {session_id}")
        print(f"Total iterations: {iteration}")
        print(f"Success: {'Yes' if debugging_session['success'] else 'No'}")
        print(f"Total time: {debugging_session['total_time']:.2f}s")
        
        if not debugging_session['success']:
            debugging_session['final_code'] = current_code  # Best attempt
            print("\n⚠️ Debugging did not fully succeed. Manual intervention may be needed.")
        
        self.debugging_sessions.append(debugging_session)
        return debugging_session
    
    def _run_tests(self, phase: str, context: Dict[str, Any], 
                   custom_tests: List[TestCase] = None) -> List[TestResult]:
        """Chạy tests cho phase hiện tại"""
        if custom_tests:
            test_cases = custom_tests
        else:
            test_cases = self.test_framework.create_data_science_test_suite(phase)
        
        results = []
        for test_case in test_cases:
            result = self.test_framework.run_test_case(test_case, context)
            results.append(result)
        
        return results
    
    def _create_error_info_from_tests(self, failed_tests: List[TestResult]) -> ErrorInfo:
        """Tạo ErrorInfo từ failed tests"""
        # Tổng hợp thông tin từ các test thất bại
        error_messages = []
        for test in failed_tests:
            if test.error:
                error_messages.append(f"{test.test_case.name}: {test.error}")
        
        combined_message = "; ".join(error_messages)
        
        return ErrorInfo(
            error_type=ErrorType.LOGIC_ERROR,
            severity=ErrorSeverity.MEDIUM,
            message=combined_message,
            traceback="Test failures",
            line_number=None,
            code_snippet=None,
            suggested_fix=None,
            timestamp=time.time(),
            context={'failed_tests': [t.test_case.name for t in failed_tests]}
        )
    
    def _extract_best_fix(self, fix_suggestions: Dict[str, Any], current_code: str) -> Optional[str]:
        """Trích xuất fix tốt nhất từ suggestions"""
        # Thử quick fix trước
        if fix_suggestions.get('quick_fix'):
            return fix_suggestions['quick_fix']
        
        # Thử extract từ LLM analysis
        if self.fix_suggester and fix_suggestions.get('llm_analysis'):
            fixed_code = self.fix_suggester.extract_fixed_code_from_analysis(
                fix_suggestions['llm_analysis']
            )
            if fixed_code:
                return fixed_code
        
        return None
    
    def get_debugging_stats(self) -> Dict[str, Any]:
        """Thống kê về các session debugging"""
        if not self.debugging_sessions:
            return {}
        
        total_sessions = len(self.debugging_sessions)
        successful_sessions = sum(1 for s in self.debugging_sessions if s['success'])
        total_iterations = sum(len(s['iterations']) for s in self.debugging_sessions)
        total_time = sum(s['total_time'] for s in self.debugging_sessions)
        
        return {
            'total_sessions': total_sessions,
            'successful_sessions': successful_sessions,
            'success_rate': successful_sessions / total_sessions * 100,
            'average_iterations': total_iterations / total_sessions,
            'average_time_per_session': total_time / total_sessions,
            'total_debugging_time': total_time
        }
    
    def export_session_report(self, session_id: str) -> Optional[Dict[str, Any]]:
        """Export chi tiết của một debugging session"""
        session = next((s for s in self.debugging_sessions if s['session_id'] == session_id), None)
        if not session:
            return None
        
        # Tạo báo cáo chi tiết
        report = {
            'session_summary': {
                'session_id': session['session_id'],
                'phase': session['phase'],
                'success': session['success'],
                'total_iterations': len(session['iterations']),
                'total_time': session['total_time']
            },
            'code_evolution': [],
            'error_patterns': [],
            'fix_effectiveness': []
        }
        
        for iteration in session['iterations']:
            report['code_evolution'].append({
                'iteration': iteration['iteration'],
                'code_length': len(iteration['code']),
                'execution_success': iteration['execution_result']['success'],
                'tests_passed': sum(1 for r in iteration.get('test_results', []) if r.passed) if iteration.get('test_results') else 0
            })
            
            if not iteration['execution_result']['success']:
                error_info = iteration['execution_result']['error_info']
                report['error_patterns'].append({
                    'iteration': iteration['iteration'],
                    'error_type': error_info.error_type.value,
                    'severity': error_info.severity.value,
                    'message': error_info.message
                })
        
        return report

# Khởi tạo debugging engine (without fix suggester for now)
debugging_engine = IterativeDebuggingEngine(
    safe_executor=safe_executor,
    test_framework=test_framework,
    fix_suggester=None,  # Would be fix_suggester if API key available
    max_iterations=3
)

print("Iterative Debugging Engine initialized successfully!")

## Phần 6: Demo Và Ví Dụ Thực Tế

Hãy demo hệ thống debugging với một số ví dụ code có lỗi phổ biến:

In [ ]:
# Demo 1: Lỗi cơ bản - KeyError
print("=== DEMO 1: KeyError Example ===")

buggy_code_1 = """
# Sample data loading with KeyError
import pandas as pd
import numpy as np

# Create sample dataframe
data = {
    'name': ['Alice', 'Bob', 'Charlie'],
    'age': [25, 30, 35],
    'salary': [50000, 60000, 70000]
}
df = pd.DataFrame(data)

# This will cause KeyError
average_income = df['income'].mean()  # Column 'income' doesn't exist
print(f"Average income: {average_income}")
"""

# Custom tests for this example
custom_tests_1 = [
    TestCase(
        name="test_dataframe_created",
        test_code="assert 'df' in globals() and hasattr(df, 'shape')",
        description="Check if DataFrame is created"
    ),
    TestCase(
        name="test_average_calculated",
        test_code="assert 'average_income' in globals()",
        description="Check if average income is calculated"
    )
]

# Run debugging session
session_1 = debugging_engine.debug_code(
    code=buggy_code_1,
    phase="data_loading",
    custom_tests=custom_tests_1
)

print(f"\nSession 1 Result: {'SUCCESS' if session_1['success'] else 'FAILED'}")

In [ ]:
# Demo 2: Lỗi Type Error
print("\n=== DEMO 2: TypeError Example ===")

buggy_code_2 = """
# Sample data processing with TypeError
import pandas as pd
import numpy as np

# Create sample data with mixed types
data = {
    'numbers': [1, 2, 3, '4', 5],  # Mixed types
    'values': [10.5, 20.1, 30.7, 40.2, 50.9]
}
df = pd.DataFrame(data)

# This will cause TypeError
total = df['numbers'].sum()  # Can't sum mixed types directly
print(f"Total: {total}")

# Convert to numeric properly
df['numbers_clean'] = pd.to_numeric(df['numbers'], errors='coerce')
total_clean = df['numbers_clean'].sum()
print(f"Clean total: {total_clean}")
"""

custom_tests_2 = [
    TestCase(
        name="test_total_calculated",
        test_code="assert 'total' in globals() or 'total_clean' in globals()",
        description="Check if total is calculated"
    ),
    TestCase(
        name="test_clean_column_created",
        test_code="assert 'numbers_clean' in df.columns",
        description="Check if clean numbers column is created"
    )
]

session_2 = debugging_engine.debug_code(
    code=buggy_code_2,
    phase="data_cleaning",
    custom_tests=custom_tests_2
)

print(f"\nSession 2 Result: {'SUCCESS' if session_2['success'] else 'FAILED'}")

In [ ]:
# Demo 3: Lỗi Logic (không crash nhưng kết quả sai)
print("\n=== DEMO 3: Logic Error Example ===")

buggy_code_3 = """
# Sample feature engineering with logic error
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Create sample dataset
np.random.seed(42)
n_samples = 100
X = np.random.randn(n_samples, 3)
y = (X[:, 0] + X[:, 1] > 0).astype(int)  # Binary target

# Convert to DataFrame
df = pd.DataFrame(X, columns=['feature1', 'feature2', 'feature3'])
df['target'] = y

# Logic error: Using target in features for training
X_train, X_test, y_train, y_test = train_test_split(
    df[['feature1', 'feature2', 'feature3', 'target']],  # BUG: including target in features
    df['target'],
    test_size=0.2,
    random_state=42
)

# Train model
model = LogisticRegression()
model.fit(X_train, y_train)

# Evaluate
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print(f"Train accuracy: {train_score:.3f}")
print(f"Test accuracy: {test_score:.3f}")
"""

custom_tests_3 = [
    TestCase(
        name="test_model_created",
        test_code="assert 'model' in globals()",
        description="Check if model is created"
    ),
    TestCase(
        name="test_no_data_leakage",
        test_code="assert X_train.shape[1] == 3, 'Features should not include target'",
        description="Check for data leakage"
    ),
    TestCase(
        name="test_reasonable_accuracy",
        test_code="assert test_score < 0.99, 'Test accuracy too high - possible data leakage'",
        description="Check for suspiciously high accuracy"
    )
]

session_3 = debugging_engine.debug_code(
    code=buggy_code_3,
    phase="model_training",
    custom_tests=custom_tests_3
)

print(f"\nSession 3 Result: {'SUCCESS' if session_3['success'] else 'FAILED'}")

In [ ]:
# Hiển thị thống kê tổng quan
print("\n" + "="*60)
print("OVERALL DEBUGGING STATISTICS")
print("="*60)

stats = debugging_engine.get_debugging_stats()
for key, value in stats.items():
    if isinstance(value, float):
        print(f"{key.replace('_', ' ').title()}: {value:.2f}")
    else:
        print(f"{key.replace('_', ' ').title()}: {value}")

# Hiển thị test results summary
print("\n" + "-"*40)
print("TEST FRAMEWORK STATISTICS")
print("-"*40)

test_report = test_framework.generate_test_report()
if not test_report.empty:
    print(f"Total tests run: {len(test_report)}")
    passed = len(test_report[test_report['Status'] == 'PASS'])
    failed = len(test_report[test_report['Status'] == 'FAIL'])
    print(f"Tests passed: {passed}")
    print(f"Tests failed: {failed}")
    print(f"Success rate: {passed/len(test_report)*100:.1f}%")
    print(f"Average execution time: {test_report['Execution Time (s)'].mean():.3f}s")
else:
    print("No test results available")

# Hiển thị execution statistics
print("\n" + "-"*40)
print("CODE EXECUTION STATISTICS")
print("-"*40)

exec_stats = safe_executor.get_execution_stats()
for key, value in exec_stats.items():
    if isinstance(value, float):
        print(f"{key.replace('_', ' ').title()}: {value:.2f}")
    else:
        print(f"{key.replace('_', ' ').title()}: {value}")

## Phần 7: Hệ Thống Learning và Knowledge Base

Cuối cùng, chúng ta sẽ tạo một hệ thống học hỏi từ các lỗi đã xử lý để cải thiện hiệu suất debug trong tương lai:

In [ ]:
class ErrorKnowledgeBase:
    """Knowledge base để lưu trữ và học hỏi từ các lỗi đã xử lý"""
    
    def __init__(self):
        self.error_patterns = []
        self.successful_fixes = []
        self.failed_fixes = []
        self.pattern_to_fix_mapping = {}
    
    def add_error_case(self, error_info: ErrorInfo, original_code: str, 
                      fix_applied: str, fix_successful: bool):
        """Thêm một case lỗi vào knowledge base"""
        case = {
            'error_info': error_info,
            'original_code': original_code,
            'fix_applied': fix_applied,
            'fix_successful': fix_successful,
            'timestamp': time.time(),
            'error_pattern': self._extract_error_pattern(error_info)
        }
        
        if fix_successful:
            self.successful_fixes.append(case)
        else:
            self.failed_fixes.append(case)
        
        # Update pattern mapping
        pattern = case['error_pattern']
        if pattern not in self.pattern_to_fix_mapping:
            self.pattern_to_fix_mapping[pattern] = []
        self.pattern_to_fix_mapping[pattern].append(case)
    
    def _extract_error_pattern(self, error_info: ErrorInfo) -> str:
        """Trích xuất pattern từ error info"""
        # Simplified pattern extraction
        pattern_elements = [
            error_info.error_type.value,
            error_info.severity.value
        ]
        
        # Extract key words from error message
        message_words = re.findall(r'\b\w+\b', error_info.message.lower())
        key_words = [word for word in message_words if len(word) > 3 and 
                    word not in ['error', 'exception', 'traceback']]
        
        if key_words:
            pattern_elements.extend(key_words[:3])  # Top 3 key words
        
        return "|".join(pattern_elements)
    
    def find_similar_cases(self, error_info: ErrorInfo, top_k: int = 5) -> List[Dict]:
        """Tìm các case tương tự đã xử lý trước đó"""
        current_pattern = self._extract_error_pattern(error_info)
        
        # Simple similarity based on pattern matching
        similar_cases = []
        
        all_cases = self.successful_fixes + self.failed_fixes
        for case in all_cases:
            similarity = self._calculate_similarity(current_pattern, case['error_pattern'])
            if similarity > 0.3:  # Threshold for similarity
                case_with_similarity = case.copy()
                case_with_similarity['similarity'] = similarity
                similar_cases.append(case_with_similarity)
        
        # Sort by similarity and success
        similar_cases.sort(key=lambda x: (x['similarity'], x['fix_successful']), reverse=True)
        
        return similar_cases[:top_k]
    
    def _calculate_similarity(self, pattern1: str, pattern2: str) -> float:
        """Tính similarity giữa hai patterns"""
        elements1 = set(pattern1.split('|'))
        elements2 = set(pattern2.split('|'))
        
        if not elements1 or not elements2:
            return 0.0
        
        intersection = elements1.intersection(elements2)
        union = elements1.union(elements2)
        
        return len(intersection) / len(union) if union else 0.0
    
    def suggest_fix_from_history(self, error_info: ErrorInfo) -> Optional[str]:
        """Đề xuất fix dựa trên lịch sử"""
        similar_cases = self.find_similar_cases(error_info)
        
        # Tìm fix thành công với similarity cao nhất
        for case in similar_cases:
            if case['fix_successful'] and case['similarity'] > 0.7:
                return case['fix_applied']
        
        return None
    
    def get_statistics(self) -> Dict[str, Any]:
        """Thống kê về knowledge base"""
        total_cases = len(self.successful_fixes) + len(self.failed_fixes)
        
        if total_cases == 0:
            return {'total_cases': 0}
        
        # Error type distribution
        error_type_counts = {}
        for case in self.successful_fixes + self.failed_fixes:
            error_type = case['error_info'].error_type.value
            error_type_counts[error_type] = error_type_counts.get(error_type, 0) + 1
        
        # Success rate by error type
        success_rates = {}
        for error_type in error_type_counts:
            successful = sum(1 for case in self.successful_fixes 
                           if case['error_info'].error_type.value == error_type)
            total = error_type_counts[error_type]
            success_rates[error_type] = successful / total * 100
        
        return {
            'total_cases': total_cases,
            'successful_fixes': len(self.successful_fixes),
            'failed_fixes': len(self.failed_fixes),
            'overall_success_rate': len(self.successful_fixes) / total_cases * 100,
            'error_type_distribution': error_type_counts,
            'success_rates_by_type': success_rates,
            'unique_patterns': len(self.pattern_to_fix_mapping)
        }
    
    def export_knowledge_base(self) -> Dict[str, Any]:
        """Export knowledge base để save/load"""
        return {
            'successful_fixes': self.successful_fixes,
            'failed_fixes': self.failed_fixes,
            'pattern_to_fix_mapping': self.pattern_to_fix_mapping
        }
    
    def load_knowledge_base(self, data: Dict[str, Any]):
        """Load knowledge base từ saved data"""
        self.successful_fixes = data.get('successful_fixes', [])
        self.failed_fixes = data.get('failed_fixes', [])
        self.pattern_to_fix_mapping = data.get('pattern_to_fix_mapping', {})

# Khởi tạo knowledge base
knowledge_base = ErrorKnowledgeBase()

# Thêm một số sample cases để demo
sample_error = ErrorInfo(
    error_type=ErrorType.KEY_ERROR,
    severity=ErrorSeverity.HIGH,
    message="KeyError: 'column_name'",
    traceback="KeyError in line 10",
    line_number=10,
    code_snippet="df['column_name']",
    suggested_fix=None,
    timestamp=time.time(),
    context={}
)

knowledge_base.add_error_case(
    error_info=sample_error,
    original_code="df['column_name']",
    fix_applied="df.get('column_name', default_value)",
    fix_successful=True
)

print("Error Knowledge Base initialized with sample data")
print("Knowledge Base Statistics:")
kb_stats = knowledge_base.get_statistics()
for key, value in kb_stats.items():
    print(f"  {key}: {value}")

## Kết Luận

Hệ thống phân tích lỗi và debugging tự động trong AutoKaggle là một thiết kế rất tiên tiến và toàn diện. Thông qua notebook này, chúng ta đã học được:

### Những Điểm Chính:

1. **Phân Loại Lỗi Có Hệ Thống**: Framework phân loại lỗi theo type và severity giúp xử lý targeted và hiệu quả

2. **Safe Code Execution**: Hệ thống thực thi code an toàn với error capturing và namespace isolation

3. **AI-Powered Error Analysis**: Sử dụng LLM để phân tích root cause và đề xuất fix strategies

4. **Automated Testing**: Unit testing framework để validate tính đúng đắn của code đã fix

5. **Iterative Debugging**: Quy trình lặp cho phép multiple attempts với learning từ previous failures

6. **Knowledge Base Learning**: Hệ thống học hỏi từ historical errors để cải thiện future debugging

### Lợi Ích Của Thiết Kế:

- **Tự Động Hóa Cao**: Giảm thiểu can thiệp manual trong quá trình debug
- **Học Hỏi Liên Tục**: Hệ thống được cải thiện qua thời gian
- **Comprehensive Testing**: Đảm bảo code quality thông qua automated testing
- **Scalability**: Có thể xử lý nhiều loại lỗi và domain khác nhau
- **Transparency**: Cung cấp detailed logging và analysis cho user

### Ứng Dụng Thực Tế:

Hệ thống này có thể được áp dụng cho:
- **Code Review Automation**: Tự động detect và suggest fixes cho common issues
- **Educational Tools**: Giúp học viên hiểu và fix errors trong code
- **Production Monitoring**: Debug runtime errors trong production environment
- **CI/CD Pipeline**: Integrate debugging vào development workflow

### Hướng Phát Triển:

1. **Advanced Pattern Recognition**: Sử dụng ML models để recognize error patterns
2. **Domain-Specific Knowledge**: Specialized debugging cho different ML domains
3. **Real-time Collaboration**: Integration với team debugging workflows
4. **Performance Optimization**: Faster debugging through parallel processing

Hệ thống error analysis và debugging trong AutoKaggle thể hiện một approach rất mature và practical cho việc tự động hóa debugging process, đặc biệt quan trọng trong context của autonomous data science workflows.